<a href="https://colab.research.google.com/github/murugadossp/Agents_Learning/blob/main/LLM_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm

['Collecting litellm',
 '  Downloading litellm-1.71.1-py3-none-any.whl.metadata (38 kB)',
 'Requirement already satisfied: aiohttp in /usr/local/lib/python3.11/dist-packages (from litellm) (3.11.15)',
 'Requirement already satisfied: click in /usr/local/lib/python3.11/dist-packages (from litellm) (8.2.0)',
 'Requirement already satisfied: httpx>=0.23.0 in /usr/local/lib/python3.11/dist-packages (from litellm) (0.28.1)',
 'Collecting httpx-aiohttp>=0.1.4 (from litellm)',
 '  Downloading httpx_aiohttp-0.1.4-py3-none-any.whl.metadata (4.5 kB)',
 'Requirement already satisfied: importlib-metadata>=6.8.0 in /usr/local/lib/python3.11/dist-packages (from litellm) (8.7.0)',
 'Requirement already satisfied: jinja2<4.0.0,>=3.1.2 in /usr/local/lib/python3.11/dist-packages (from litellm) (3.1.6)',
 'Requirement already satisfied: jsonschema<5.0.0,>=4.22.0 in /usr/local/lib/python3.11/dist-packages (from litellm) (4.23.0)',
 'Requirement already satisfied: openai>=1.68.2 in /usr/local/lib/python3.1

In [7]:


# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [8]:
import json
import os
from typing import List

from litellm import completion

In [27]:
def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

def terminate(message: str) -> None:
    """Terminate the agent loop and provide a summary message."""
    print(f"Termination message: {message}")

In [28]:
tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate
}

In [29]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},
                },
                "required": ["message"]
            }
        }
    }
]

In [30]:
# Our rules are simplified since we don't have to worry about getting a specific output format
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

In [38]:
def user_task():

  # Initialize agent parameters
  iterations = 1
  max_iterations = 10

  user_task = input("What would you like me to do? ")

  memory = [{"role": "user", "content": user_task}]

  # The Agent Loop
  while iterations <= max_iterations:

      messages = agent_rules + memory

      response = completion(
          model="openai/gpt-4o",
          messages=messages,
          tools=tools,
          max_tokens=1024
      )

      if response.choices[0].message.tool_calls:
          tool = response.choices[0].message.tool_calls[0]
          tool_name = tool.function.name
          tool_args = json.loads(tool.function.arguments)

          action = {
              "tool_name": tool_name,
              "args": tool_args
          }
          print(f"Iteration {iterations}: LLM Action output {action}")

          if tool_name == "terminate":
              print(f"Termination message: {tool_args['message']}")
              break
          elif tool_name in tool_functions:

              try:
                  result = {"result": tool_functions[tool_name](**tool_args)}
              except Exception as e:
                  result = {"error":f"Error executing {tool_name}: {str(e)}"}
          else:
              result = {"error": f"Unknown tool: {tool_name}"}

          print(f"Executing: {tool_name} with args {tool_args}")
          print(f"Result: {result}")
          memory.extend([
              {"role": "assistant", "content": json.dumps(action)},
              {"role": "user", "content": json.dumps(result)}
          ])
      else:
          result = response.choices[0].message.content
          print(f"Response: {result}")
          break
      iterations += 1

In [39]:
user_task()

What would you like me to do? read the contents of test.txt
Iteration 1: LLM Action output {'tool_name': 'list_files', 'args': {}}
Executing: list_files with args {}
Result: {'result': ['.config', 'test.txt', 'sample_data']}
Iteration 2: LLM Action output {'tool_name': 'read_file', 'args': {'file_name': 'test.txt'}}
Executing: read_file with args {'file_name': 'test.txt'}
Result: {'result': 'Line1\nLine2\nLine3\n'}
Iteration 3: LLM Action output {'tool_name': 'terminate', 'args': {'message': 'The content of test.txt is:\nLine1\nLine2\nLine3'}}
Termination message: The content of test.txt is:
Line1
Line2
Line3


In [41]:
# Create and write to the file 'test.txt'
# Using "w" mode creates the file if it doesn't exist
with open("test2.txt", "w") as fp:
  fp.writelines(["test2 Line1\n", "test2 Line2\n" , "test3 Line3\n"]) # Added newline characters for proper line breaks

# Now you can call user_task() or perform other operations that might read this file
user_task()

What would you like me to do? read all the file contents
Iteration 1: LLM Action output {'tool_name': 'list_files', 'args': {}}
Executing: list_files with args {}
Result: {'result': ['.config', 'test.txt', 'test2.txt', 'sample_data']}
Iteration 2: LLM Action output {'tool_name': 'read_file', 'args': {'file_name': 'test.txt'}}
Executing: read_file with args {'file_name': 'test.txt'}
Result: {'result': 'Line1\nLine2\nLine3\n'}
Iteration 3: LLM Action output {'tool_name': 'read_file', 'args': {'file_name': 'test2.txt'}}
Executing: read_file with args {'file_name': 'test2.txt'}
Result: {'result': 'test2 Line1\ntest2 Line2\ntest3 Line3\n'}
Iteration 4: LLM Action output {'tool_name': 'terminate', 'args': {'message': 'Here are the contents of the files:\n\n- **test.txt**:\n  Line1\n  Line2\n  Line3\n\n- **test2.txt**:\n  test2 Line1\n  test2 Line2\n  test3 Line3\n\nIf you need further assistance, feel free to ask!'}}
Termination message: Here are the contents of the files:

- **test.txt**:
 